In [66]:
import os
import keras
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras import layers
from keras import models
from keras import Input
from keras import preprocessing
from keras.src.utils import img_to_array
from sklearn.model_selection import train_test_split

In [67]:
# Paths to data
gunshot_path = "../data/processed/gunshot"
non_gunshot_path = "../data/processed/not-gunshot"

# Verify that paths exists
print(os.path.exists(gunshot_path))
print(os.path.exists(non_gunshot_path))

True
True


In [68]:
# Function to load and process images -> 0 for non-gunshot and 1 for gunshot
def load_images(path, label):
    images = []
    labels = []

    for filename in os.listdir(path):

        # Only accept images
        valid_extensions = ['.png', '.jpg', '.jpeg']
        if not any(filename.lower().endswith(ext) for ext in valid_extensions):
            continue

        # Load local path to image
        img_path = os.path.join(path, filename)

        print(f"Attempting to load image {img_path}")

        try:
            # Resize to (128, 128) RGBA pixel image
            img = preprocessing.image.load_img(img_path, target_size=(128, 128), color_mode="rgba")

            # Convert RGBA to RGB
            img = img.convert("RGB")

            # Flatten into numpy array
            img = img_to_array(img)

            # Append image and corresponding label
            images.append(img)
            labels.append(label)
        except Exception as error:
            print(error)
            continue

    return images, labels

In [69]:
# Load gunshots
gunshot_images, gunshot_labels = load_images(gunshot_path, 1)
non_gunshot_images, non_gunshot_labels = load_images(non_gunshot_path, 0)

Attempting to load image ../data/processed/gunshot/gunshot_4.png
Attempting to load image ../data/processed/not-gunshot/gunshot_1.png
Attempting to load image ../data/processed/not-gunshot/gunshot_2.png
Attempting to load image ../data/processed/not-gunshot/gunshot_3.png


In [70]:
# Combine gunshots and labels of both types
images = gunshot_images + non_gunshot_images
labels = gunshot_labels + non_gunshot_labels

# Convert them to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize images
images = images / 255.0

In [71]:
# 20% for testing 80# for training
# X_train -> training subset for input features (images)
# X_val   -> validation subset for input features (images)
# y_train -> training subset for labels
# y_val   -> validation subset for labels
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [77]:
# Define the CNN model
model = models.Sequential()

# Input layer to define the input shape
# Output -> 3D Tensor shape (128, 128, 3) -> RGB image 128x128 pixels
model.add(Input(shape=(128, 128, 3)))

# Applies 32 convolution filters of dimension 3x3 to input
# Output -> (126, 126, 32)
model.add(layers.Conv2D(32, (3, 3), activation='relu'))

# Reduces spatial dimensions by taking max value within overlapping 2x2 regions
# Output -> (63, 63, 32)
model.add(layers.MaxPooling2D((2, 2)))

# Applies 64 convolution filters of dimension 3x3 to input
# Output -> (61, 61, 64)
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Reduces spatial dimensions by taking max value within overlapping 2x2 regions
# Output -> (30, 30, 64)
model.add(layers.MaxPooling2D((2, 2)))

# Applies 128 convolution filters of dimension 3x3 to input
# Output -> (28, 28, 128)
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

# Reduces spatial dimensions by taking max value within overlapping 2x2 regions
# Output -> (14, 14, 128)
model.add(layers.MaxPooling2D((2, 2)))

# Converts 3D output from previous layer to 1D vector
# Output -> (25088, )
model.add(layers.Flatten())

# Applies 128 neurons to learn 'high level' features
# Output -> (128, )
model.add(layers.Dense(128, activation='relu'))

# Sigmoid function to determine gunshot or non-gunshot
# 0 = non-gunshot, 1 = gunshot
model.add(layers.Dense(1, activation='sigmoid'))


In [73]:
# Compile the model:
# Optimizer -> adam
# loss function -> binary_crossentropy
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [74]:
# Train the model:
# Epochs -> Total passes through model
# Batch size -> # of samples the model processes before updating weights
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 696ms/step - accuracy: 0.3333 - loss: 0.7091 - val_accuracy: 1.0000 - val_loss: 0.0508
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.6667 - loss: 0.6371 - val_accuracy: 1.0000 - val_loss: 0.1483
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 0.2497 - val_accuracy: 1.0000 - val_loss: 0.1486
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 1.0000 - loss: 0.1798 - val_accuracy: 1.0000 - val_loss: 0.0666
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 1.0000 - loss: 0.0866 - val_accuracy: 1.0000 - val_loss: 0.0145
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 0.0294 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 0.0084 - val_accuracy: 1.0000 - val_loss: 1.5537e-04
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 1.

In [75]:
# Determine performance metrics
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Test accuracy: {test_acc}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 5.8997e-08
Test accuracy: 1.0


In [79]:
# Save model
model_name = "test_model"
model.save(f'../models/{model_name}.h5')

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=../models/test_model.